In [2]:
import numpy as np
import matplotlib.pyplot as plt
import hybrideb #https://github.com/beckermr/hybrideb
from tqdm import tqdm

In [ ]:
import hybrideb

theta_min = 2.5 # arcmin
theta_max = 250 # arcmin
Ntheta = 1000 # number of bins in log(theta)

heb = hybrideb.HybridEB(theta_min, theta_max, Ntheta)
beb = hybrideb.BinEB(theta_min, theta_max, Ntheta)

geb = hybrideb.GaussEB(beb, heb)

res = geb(3)  # grab the third estimator
theta_rad = res[0]
# X+ = np.sum((fp*xip + fm*xim)/2)
# X- = np.sum((fp*xip - fm*xim)/2)
fp = res[1]
fm = res[2]

# X+ = \int ell factors(ell) (wp * Pe + wm * Pb)
# X- = \int ell factors(ell) (wm * Pe + wp * Pb)
ell = res[3]
wp = res[4]
wm = res[5]

In [ ]:
np.save('EB_Coefficients_DESscales', np.array([geb(i) for i in range(20)]))

In [ ]:
plt.yscale('log')
plt.xscale('log')
x = np.loadtxt('Shear_treecorr.txt')

plt.yticks([], [])

plt.plot(x[:, 0], x[:, 3])

plt.xlabel('arcmin', size = 20)
plt.ylabel('Nope  :)', size = 20)
plt.title('xi+', size = 20)

In [ ]:
#Plot the coefficients
for i in range(20):
    
    plt.figure(figsize = (12,3))
    plt.xscale('log')
    res = geb(i)
    plt.plot(res[0] * 180/np.pi * 60, res[2])
    plt.plot(res[0] * 180/np.pi * 60, res[1])
    plt.show()

In [ ]:
x = np.loadtxt('shear_treecorr.txt')

d = np.concatenate([x[:, 3], x[:, 4]])
cov = np.loadtxt('shear_treecorr.txt')

x = np.random.multivariate_normal(mean = d, cov = cov, size = 1000)


xip_MC = x[:, :1000]
xim_MC = x[:, 1000:]


En = np.zeros([xip_MC.shape[0], 20])
Bn = np.zeros([xip_MC.shape[0], 20])

mean_ell = np.zeros(20)

geb_saved = np.load('EB_Coefficients.npy', allow_pickle = True)

for j in tqdm(range(xip_MC.shape[0])):
    for i in range(20):

        res = geb_saved[i]

        # X+ = np.sum((fp*xip + fm*xim)/2)
        # X- = np.sum((fp*xip - fm*xim)/2)
        fp = res[1]
        fm = res[2]
        
        Mask = (res[0]*180/np.pi * 60 < 250) & (res[0]*180/np.pi * 60 > 2.5)
        
#         fp[~Mask] = 0
#         fm[~Mask] = 0

        En[j, i] = np.sum(fp*xip_MC[j] + fm*xim_MC[j])/2
        Bn[j, i] = np.sum(fp*xip_MC[j] - fm*xim_MC[j])/2
        
        if j == 0:
            mean_ell[i] = np.sum(res[3]*res[4])/np.sum(res[4])

En = np.array(En)
Bn = np.array(Bn)

In [ ]:
plt.fill_between(range(20), np.percentile(Bn, 16, 0), np.percentile(Bn, 84, 0), alpha = 0.4, label = 'X-')
plt.fill_between(range(20), np.percentile(En, 16, 0), np.percentile(En, 84, 0), alpha = 0.4, label = 'X+')
plt.yticks([], [])
plt.xticks(np.arange(0, 20, 2), np.arange(0, 20, 2));
plt.axhline(0, color = 'k')
plt.legend(fontsize = 18)

plt.xlabel('Mode index, i', size = 20)
plt.ylabel('Nope  :)', size = 20)

cov = np.cov(Bn.T)
inv_cov = np.linalg.inv(cov)

diff = np.mean(Bn, axis = 0)

chi2 = diff @ inv_cov @ diff

plt.title(r'$\chi^2/N_{\rm dof} = %0.1f/%d \,\,(%0.2f\sigma)$' % (chi2, Bn.shape[1], chi2/Bn.shape[1]), size = 20)